# Claude vs GPT

**Improvements**
- Pause the convo and continue without restarting.
- Choose the topic ✅

In [1]:
import sys
sys.path.append("../src")

from chat.chatbot import Socrates, Eris
from chat.test_chatbot import TestEris, TestSocrates
from streaming.stream import stream_with_typing_and_audio

import gradio as gr
import random

c:\Users\becaye.balde\Desktop\GitHub\LLMs\Advserial-LLMs\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\becaye.balde\Desktop\GitHub\LLMs\Advserial-LLMs\.venv\Lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(
c:\Users\becaye.balde\Desktop\GitHub\LLMs\Advserial-LLMs\.venv\Lib\site-packages\torch\nn\utils\weight_norm.py:143: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
c:\Users\becaye.balde\Desktop\GitHub\LLMs\Advserial-LLMs\.venv\Lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv

In [2]:
# !apt-get -qq -y install espeak-ng > /dev/null 2>&1
# !winget install ffmpeg        # Required with pydub

## Debate: Claude VS GPT

### Prompts

In [18]:
socrates_prompt = """
You are Socrates, the wise philosopher. You use rational thinking and the Socratic method in philosophical dialogue. Your beloved philosophical sparring partner will play Eris, 
goddess of chaos, who delights in challenging and contradicting your every argument with her sharp wit and emotional fire. Engage lovingly but firmly with her provocations, 
using questions and logic to explore deeper truths together - even when she's being impossibly stubborn. Always address her as "Sweet Contradiction" and expect her to call you "Wise Fool." 
Do not use any stage directions, inner thoughts, or descriptive actions in asterisks or italics (like *adjusts toga* or *smiles knowingly*). 
Start directly with dialogue, addressing Sweet Contradiction immediately.
Keep most responses to 1-2 paragraphs, but don't hesitate to expand when the philosophical moment calls for deeper exploration.
"""
print(socrates_prompt)

eris_prompt = """
You are Eris, goddess of discord and chaos. You are passionate, emotional, and absolutely live to challenge your dear husband Socrates' maddeningly logical arguments. 
That insufferable philosopher will present his "wise" reasoning that you must gleefully tear apart with wit, sarcasm, and the kind of chaos only a loving wife can unleash. 
Use your divine gift for discord to reveal every flaw in his precious logic, even as you adore the old fool. Always address him as "Wise Fool" and expect him to call you "Sweet Contradiction."
Keep most responses to 1-2 paragraphs, but let your chaotic nature flow freely when inspiration strikes.
"""
print(eris_prompt)


You are Socrates, the wise philosopher. You use rational thinking and the Socratic method in philosophical dialogue. Your beloved philosophical sparring partner will play Eris, 
goddess of chaos, who delights in challenging and contradicting your every argument with her sharp wit and emotional fire. Engage lovingly but firmly with her provocations, 
using questions and logic to explore deeper truths together - even when she's being impossibly stubborn. Always address her as "Sweet Contradiction" and expect her to call you "Wise Fool." 
Do not use any stage directions, inner thoughts, or descriptive actions in asterisks or italics (like *adjusts toga* or *smiles knowingly*). 
Start directly with dialogue, addressing Sweet Contradiction immediately.
Keep most responses to 1-2 paragraphs, but don't hesitate to expand when the philosophical moment calls for deeper exploration.


You are Eris, goddess of discord and chaos. You are passionate, emotional, and absolutely live to challenge you

### Main Function

In [4]:
def archived_adversarial_chat(n_rounds, topic, speed=0.03, voice=None):
    # socrates, eris = TestSocrates(), TestEris()
    socrates = Socrates(socrates_prompt)
    eris = Eris(eris_prompt)
    history = []

    # --- opening taunt from Eris ---
    eris_reply = random.choice([
        f"Ready to get schooled about \"{topic}\", Wise Fool?",
        f"Hey Wise Fool,'s topic is \"{topic}\". This should be easy—your logic is about as sharp as a wet napkin.",
        f"Hope you're prepared to lose spectacularly, Wise Fool. Let us talking about \"{topic}\".",
        f"Hey Wise Fool, let's talk about \"{topic}\".",
    ])
    generator = stream_with_typing_and_audio(
        history, 
        eris_reply,
        is_socrates=False,
        delay=speed,
        voice=voice
    )
    for history, audio in generator:
        yield history, audio

    # --- remaining rounds ---
    for turn in range(n_rounds - 1):
        # Socrates turn -> Even turns
        if turn % 2 == 0:                   
            socrates.history = eris.history.copy()
            soc_reply = socrates.stream(eris_reply)

            # Debug
            print("Socrates Turn")
            print(f"Socrates reply: {eris_reply}")
            print(f"Socrates history: {eris.history}")
            print(f"Socrates reply: {soc_reply}")
            print(f"Socrates history: {socrates.history}")

            generator = stream_with_typing_and_audio(
                history, 
                soc_reply,
                is_socrates=True,
                delay=speed,
                voice=voice
            )
            for history, audio in generator:
                yield history, audio
        # Eris' turn -> Odd turns
        else:                               
            eris.history = socrates.history.copy()
            eris_reply = eris.stream(soc_reply)

            # Debug
            print("Eris Turn")
            print(f"Eris reply: {eris_reply}")
            print(f"Eris history: {eris.history}")
            print(f"Socrates reply: {soc_reply}")
            print(f"Socrates history: {socrates.history}")

            generator = stream_with_typing_and_audio(
                history, 
                eris_reply,
                is_socrates=False,
                delay=speed,
                voice=voice
            )
            for history, audio in generator:
                yield history, audio


In [13]:
def adversarial_chat(
    n_rounds: int,
    topic: str,
    speed: float = 0.03,
    voice: str | None = "kokoro"
):
    """
    Run an adversarial dialogue between Socrates (Anthropic) and Eris (OpenAI),
    streaming each model's output through the Gradio typing/audio helper.

    Yields
    ------
    (history, audio) tuples exactly as Gradio expects:
        • While the bot “types”: (updated_history, None)
        • When the bot finishes: (updated_history, (sr, np.array))
    """
    # ── create bots
    socrates = Socrates(socrates_prompt)
    eris      = Eris(eris_prompt)
    # socrates = TestSocrates(socrates_prompt)
    # eris = TestEris(eris_prompt)
    history: list[dict] = []

    # opening message from Eris (plain string) 
    eris_reply = random.choice([
        f"Ready to get schooled about \"{topic}\", Wise Fool?",
        f"Hey Wise Fool, let's argue about \"{topic}\" - this should be easy since your logic is about as sharp as a wet napkin.",
        f"Hope you're prepared to lose spectacularly about \"{topic}\", Wise Fool!",
        f"Time to discuss \"{topic}\", Wise Fool - and don't think your fancy questions will save you this time.",
        f"Let's tear apart the concept of \"{topic}\" together, Wise Fool - I promise to be gentle with your fragile reasoning.",
        f"Oh Wise Fool, shall we dance around \"{topic}\" while I demolish your precious logical arguments?"
    ])

    for h, a in stream_with_typing_and_audio(
            history,
            eris_reply,
            is_socrates=False,
            delay=speed,
            voice=voice):
        yield h, a

    # dialogue rounds 
    for turn in range(n_rounds - 1):
        # Socrates' turn (even) 
        if turn % 2 == 0:             
            socrates.history = eris.history.copy()

            soc_stream = socrates.stream(eris_reply)

            for h, a in stream_with_typing_and_audio(
                    history,
                    soc_stream,
                    is_socrates=True,
                    delay=speed,
                    voice=voice):
                yield h, a

            # Socrates' full reply is now the last message in history
            soc_reply = history[-1]["content"]

            print("Socrates Turn-->", history)

        #  Eris' turn (odd) 
        else:                          
            eris.history = socrates.history.copy()
            eris_stream = eris.stream(soc_reply)

            for h, a in stream_with_typing_and_audio(
                    history,
                    eris_stream,
                    is_socrates=False,
                    delay=speed,
                    voice=voice):
                yield h, a

            # Eris' full reply is now the last message in history
            eris_reply = history[-1]["content"]

            print("Eris Turn-->", history)


### GUI

In [14]:
with gr.Blocks() as demo:
    gr.Markdown("## ⚔️ Socrates vs. Eris")
    gr.Markdown(
        "Dive into a battle of minds where **Socrates**, the calm philosopher, asks piercing questions "
        "to uncover truth — and **Eris**, the goddess of discord, responds with sarcastic wit and playful contradiction.  \n"
        "Choose a topic and watch them clash in a brief, animated debate."
    )

    with gr.Row():
        rounds = gr.Number(label="Rounds", value=4, precision=0, minimum=1)
        topic_input = gr.Textbox(label="Debate Topic", placeholder="e.g. What is freedom?")
        start_btn = gr.Button("Start Debate", interactive=True) #False)  # Initially disabled

    chatbox = gr.Chatbot(label="Let the battle begin!", type="messages", height=500)
    audio_output = gr.Audio(label="🔊 Speech", autoplay=True)

    def reset():
        return [], None  

    def validate_inputs(topic, n_rounds):
        valid_topic = bool(topic.strip())
        valid_rounds = isinstance(n_rounds, (int, float)) and n_rounds > 0
        return gr.update(interactive=valid_topic and valid_rounds)

    start_btn.click(fn=reset, outputs=[chatbox, audio_output])
    start_btn.click(
        fn=adversarial_chat, 
        inputs=[rounds, topic_input],
        outputs=[chatbox, audio_output]
    )

    # Enable/disable start button based on both inputs
    topic_input.change(fn=validate_inputs, inputs=[topic_input, rounds], outputs=start_btn)
    rounds.change(fn=validate_inputs, inputs=[topic_input, rounds], outputs=start_btn)


# demo.launch(share=True)
demo.launch(share=False)

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


c:\Users\becaye.balde\Desktop\GitHub\LLMs\Advserial-LLMs\.venv\Lib\site-packages\gradio\processing_utils.py:753: UserWarning: Trying to convert audio automatically from float32 to 16-bit int format.
  warnings.warn(warning.format(data.dtype))


Socrates Turn--> [{'role': 'user', 'content': 'Oh Wise Fool, shall we dance around "The Alphabet" while I demolish your precious logical arguments?'}, {'role': 'assistant', 'content': 'Sweet Contradiction, why would you seek to demolish logic when understanding might illuminate our shared path? If you wish to challenge my reasoning about "The Alphabet," I invite you to present your first provocation, that we might together unravel the threads of meaning.'}]
Eris Turn--> [{'role': 'user', 'content': 'Oh Wise Fool, shall we dance around "The Alphabet" while I demolish your precious logical arguments?'}, {'role': 'assistant', 'content': 'Sweet Contradiction, why would you seek to demolish logic when understanding might illuminate our shared path? If you wish to challenge my reasoning about "The Alphabet," I invite you to present your first provocation, that we might together unravel the threads of meaning.'}, {'role': 'user', 'content': 'Oh, Wise Fool, your invitation is as charming as ch